# Libraries

In [5]:
import subprocess
import json
import requests
import pandas as pd
import codecs
from datetime import date
import pymysql
today = date.today().strftime("%Y-%m-%d")

# First: this is the part where I authenticate myself to the API 

In [6]:
def get_token():
    cmd = """curl https://api.yusofleet.com/api/v3/back_end_user/authenticate --ssl-no-revoke -H \"Content-Type:application/json\" -d "{\\"saas_office_name\\":\\"Lamma-city\\",\\"email\\":\\"saief@lamma.city\\",\\"password\\":\\"SaiefT511044.\\"}\""""
    result = subprocess.run(cmd, stdout=subprocess.PIPE, shell = True)
    return json.loads(str(result.stdout).split("\'")[1])['jwt']

# Second: here is where we generate the UUID of the data needed

In [7]:
def generate_data_export(JWT, table_name, start, end):
    cmd = """curl -X POST https://api.yusofleet.com/api/v3/back_office/Lamma/Lamma-city/data_exports --ssl-no-revoke -H \"Authorization: Bearer {}\" -H \"Content-Type: application/json\" -d \"{{\\"dataclip_key\\":\\"{}\\", \\"start_date\\": \\"{}\\", \\"end_date\\": \\"{}\\"}}\"""".format(JWT, table_name, start, end)
    result = subprocess.run(cmd, stdout=subprocess.PIPE, shell = True)
    r = json.loads(str(result.stdout).split("\'")[1].replace('\\',''))
    col_name = "uuid" if "uuid" in r else "export_uuid"
    return r[col_name]

# Third: in this part we generate the final URL of the data

In [8]:
def generate_temporary_link(JWT, UUID):
    cmd = """curl -X GET \"https://api.yusofleet.com/api/v3/back_office/Lamma/Lamma-city/data_exports/{}\" --ssl-no-revoke -H \"Authorization: Bearer {}\" """.format(UUID, JWT)
    result = subprocess.run(cmd, stdout=subprocess.PIPE, shell = True)
    return json.loads(str(result.stdout).split("\'")[1])['url'].replace('\\u0026', '&')

# Finally, I combine those previous three functions into one, so I can finally generate the data I need - It will come in a text type

In [9]:
def get_data():
    jwt = get_token()
    uuid = generate_data_export(jwt, "saas_office_connexion_time", '2020-11-05', today)
    url = generate_temporary_link(jwt, uuid)
    r = requests.get(url, allow_redirects=True)
    content = r.content.decode("utf-8-sig","ignore")
    #print(type(content)) #string
    #print(content)
    
    #df = pd.DataFrame(content)
    #df.head()
    
    # the return of this function is a text output 
    # of the data 'connexion time' from 30 october till today
    return content

In [1]:
#get_data()

# Final

In [10]:
def data_to_sql():
    
    conn = pymysql.connect( host='127.0.0.1', user='root', passwd='', db='lamma' )
    #query = ("CREATE TABLE data_lamma (id_vhicule varchar(255), primaire_secondaire varchar(255), working_status varchar(255), chauffeur varchar(255), statut varchar(255), date varchar(255), temps varchar(255));")
    query = ("INSERT INTO data_lamma(id_vhicule, primaire_secondaire, working_status, chauffeur, statut, date, temps)" "VALUES (%s, %s, %s, %s, %s, %s, %s)")

    def ExeQr(conn, query, data) :
            cur = conn.cursor()
            cur.execute(query, data)
    
    content = get_data() 
    lines = content.split('\n')
    header = ['Id du type de vhicule','Primaire/Secondaire','Working status','Id du chauffeur','Statut','Date','Temps (mn)']    

    for line in lines[1:-2] :
        col = line.split(';')

        id_vhicule = str(col[0])
        primaire_secondaire = str(col[1])
        working_status = str(col[2])
        chauffeur = str(col[3])
        statut = str(col[4])
        date = str(col[5])
        temps = str(col[6])   
        
        data = (id_vhicule, primaire_secondaire, working_status, chauffeur, statut, date, temps)
        
        ExeQr(conn, query, data)
        
    conn.close()      

In [11]:
data_to_sql()